# Hébergement local de modèles génératifs


## 1) Installation & Import

On installe/importe ce qui est nécessaire : 
- `requests` pour les appels HTTP bruts,
- `openai` version 1.0.0+,
- `semantic-kernel` si on veut tester SK,
- d’autres libs selon besoin (json, time, etc.).



In [ ]:
%pip install --upgrade requests aiohttp openai semantic-kernel python-dotenv anyio httpx httpcore


import os
import requests
import time
import json
import getpass
import openai


print("Importations OK.")

## Journalisation colorée

Nous allons utiliser un logger (via le module `logging`) configuré avec un
**ColorFormatter** pour afficher les messages en couleur dans la console ou la
sortie de Jupyter :

- Les **informations** et étapes réussies apparaîtront en vert (niveau `INFO`).
- Les **erreurs** seront en rouge (niveau `ERROR`).
- Les avertissements (`WARNING`) ou messages de debug (`DEBUG`) auront également leurs couleurs.


In [ ]:
import logging
class ColorFormatter(logging.Formatter):
    """
    Un formatter coloré pour rendre les logs plus lisibles.
    """
    colors = {
        'DEBUG': '\033[94m',
        'INFO': '\033[92m',
        'WARNING': '\033[93m',
        'ERROR': '\033[91m',
        'CRITICAL': '\033[91m\033[1m'
    }
    reset = '\033[0m'

    def format(self, record: logging.LogRecord) -> str:
        msg = super().format(record)
        return f"{self.colors.get(record.levelname, '')}{msg}{self.reset}"

logger = logging.getLogger("Orchestration")
logger.setLevel(logging.DEBUG)  # Peut être paramétré via .env ou variable

if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    formatter = ColorFormatter(
        fmt="%(asctime)s [%(levelname)s] %(name)s - %(message)s",
        datefmt="%H:%M:%S"
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)

logger.info("Configuration initiale terminée.")

## Configuration et définition dynamique des endpoints

Pour simplifier la configuration de nos endpoints (URL d’API, clés d’API, modèles, etc.), nous allons externaliser ces informations dans un fichier `.env` placé à la racine de notre projet ou dans un dossier sécurisé. Copiez le fichier .env.example et renommez le fichier résultant en .env avant de le personnaliser.

### 1) Déclaration dans `.env`

On déclare, par exemple, un premier endpoint dans des variables d’environnement :

```
OPENAI_ENDPOINT_NAME=OpenAI
OPENAI_BASE_URL=https://api.openai.com/v1
OPENAI_API_KEY=sk-abcd1234ABCD1423
OPENAI_CHAT_MODEL_ID=gpt-4o-mini
```

Et si l’on souhaite tester plusieurs endpoints (ex. mini, medium, large), on ajoute un suffixe `_2`, `_3`... :

```
OPENAI_ENDPOINT_NAME_2=local-mini
OPENAI_BASE_URL_2=https://api.mini.yourdomain.com/v1
OPENAI_API_KEY_2=sk-MINI-SECRET-KEY
OPENAI_CHAT_MODEL_ID_2=unsloth/DeepSeek-R1-Distill-Llama-8B-bnb-4bit

OPENAI_ENDPOINT_NAME_3=medium
OPENAI_BASE_URL_3=https://api.medium.text-generation-webui.myia.io/v1
OPENAI_API_KEY_3=sk-MEDIUM-SECRET-KEY
OPENAI_CHAT_MODEL_ID_3=unsloth/DeepSeek-R1-Distill-Qwen-14B-bnb-4bit
```

Ainsi, chacun de ces blocs définit un **endpoint** : un service local ou distant OpenAI-compatible (ex. Oobabooga ou vLLM).  

### 2) Lecture et création automatique dans le notebook

Dans le notebook, nous allons **lire** ces variables pour construire la liste `endpoints`. Chacun contient :

- `name` : un label descriptif (ex. `micro`, `mini`, etc.),
- `api_base` : l’URL de base de l’API (ex. `https://api.micro.text-generation-webui.myia.io/v1`),
- `api_key` : la clé API (fournie par votre conteneur ou config),
- `model` (optionnel) : si le modèle n’est pas fourni, nous pourrons interroger `/models` pour récupérer le nom du (ou des) modèle(s) disponibles.

Grâce à cette configuration dynamique, on peut aisément **alterner** entre différents backends (p. ex. Oobabooga ou vLLM) ou **interroger plusieurs endpoints** pour **comparer leurs performances**.

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # Charge automatiquement toutes les variables du fichier .env


import os

def get_optional_env(var_name, default_value=None):
    """
    Récupère la valeur de la variable d'env `var_name`, 
    ou la valeur par défaut `default_value` si non définie.
    """
    val = os.getenv(var_name)
    if val is None or val.strip() == "":
        return default_value
    return val.strip()

def load_endpoint(index=1):
    """
    Lit un ensemble de variables d'environnement.
    - index=1 => variables : OPENAI_API_KEY, OPENAI_BASE_URL, etc.
    - index>1 => on suffixe : OPENAI_API_KEY_{index}, etc.
    Retourne un dict {name, api_base, api_key, model} ou None si 'api_key' manquant.
    """
    suffix = "" if index == 1 else f"_{index}"

    # Lecture des variables
    api_key = os.getenv(f"OPENAI_API_KEY{suffix}")
    if not api_key:
        return None  # pas de clé => on arrête

    name = get_optional_env(f"OPENAI_ENDPOINT_NAME{suffix}", default_value=f"openai{suffix}")
    base_url = get_optional_env(f"OPENAI_BASE_URL{suffix}", default_value="https://api.openai.com/v1")
    model_id = get_optional_env(f"OPENAI_CHAT_MODEL_ID{suffix}", default_value=None)

    return {
        "name": name,
        "api_base": base_url,
        "api_key": api_key,
        "model": model_id  # On pourra le compléter si None
    }


endpoints = []

# On tente successivement index=1,2,3... jusqu'à ce qu'on ne trouve plus OPENAI_API_KEY_{i}
for i in range(1, 10):  # max 9 endpoints, ajustez si besoin
    ep = load_endpoint(i)
    if ep is None:
        break
    endpoints.append(ep)

# Vérification (simple)
logger.info("=== Endpoints chargés ===")
for e in endpoints:
    logger.info(f"- name={e['name']}, base={e['api_base']}, key=(len={len(e['api_key'])}), model={e['model']}")



## 3) Inspection des modèles disponibles

Nous allons appeler l'endpoint `/models` de chaque service 
pour récupérer la liste des modèles chargés côté serveur.

Si vous avez mis `model=None` dans la config, vous pourrez automatiquement 
récupérer le `model` à partir des données renvoyées. 


In [ ]:

def list_models(api_base, api_key):
    url = f"{api_base}/models"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    try:
        resp = requests.get(url, headers=headers, timeout=20)
        if resp.status_code == 200:
            return resp.json()  # dict
        else:
            return {"error": f"status={resp.status_code}", "text": resp.text}
    except Exception as e:
        return {"error": str(e)}

def update_endpoints_with_model():
    for ep in endpoints:
        logger.info(f"=== {ep['name']} : /models ===")
        start_time = time.time()

        info = list_models(ep["api_base"], ep["api_key"])
        elapsed_time = time.time() - start_time

        # On journalise le JSON complet en DEBUG, pour le diagnostic
        logger.debug(f"Réponse brute: {info}")

        if "error" in info:
            # En cas d'erreur, on logue au niveau ERROR
            logger.error(
                f"Échec de récupération /models "
                f"(endpoint={ep['name']}): {info['error']}, "
                f"texte={info.get('text', '')}"
            )
        else:
            # Succès : on peut afficher le nombre de modèles
            data_models = info.get("data", [])
            logger.info(
                f"Réussite: {len(data_models)} modèle(s) listé(s) "
                f"(endpoint={ep['name']})"
            )

        logger.info(f"  -> Temps de réponse: {elapsed_time:.2f} secondes")

        # On met à jour le modèle si:
        # 1) il n'y a pas d'erreur, et
        # 2) ep["model"] n'est pas déjà défini.
        if "error" not in info and ("model" not in ep or not ep["model"]):
            data_list = info.get("data", [])
            if data_list:
                first_model_id = data_list[0].get("id")
                ep["model"] = first_model_id
                logger.info(f"  -> ep['model'] défini à: {first_model_id}")
            else:
                logger.warning(
                    "  -> Aucune entrée 'data' dans la réponse pour définir ep['model']"
                )

# Appel de la fonction pour tester
update_endpoints_with_model()


## 4) Test brut via `requests.post`

Ce test vérifie le bon fonctionnement de l'endpoint OpenAI-compatible 
sans passer par la librairie `openai`. 

On envoie une requête minimaliste en JSON, 
puis on affiche la réponse brute.


In [ ]:
def test_brut_endpoints():
    """Test brut via requests.post() sur tous les endpoints."""
    for ep in endpoints:
        logger.info(f"\n=== Test HTTP brut pour {ep['name']} ===")
        
        url = f"{ep['api_base']}/chat/completions"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {ep['api_key']}"
        }
        payload = {
            "model": ep["model"],
            "messages": [
                {"role": "user", "content": "Bonjour, qui es-tu ?"}
            ],
            "max_completion_tokens": 50
        }
        
        logger.debug("  -> Envoi de la requête POST...")
        start_time = time.time()
        try:
            resp = requests.post(url, headers=headers, json=payload, timeout=30)
            elapsed_time = time.time() - start_time
            logger.debug(f"  -> Statut HTTP: {resp.status_code} (durée={elapsed_time:.2f}s)")
            
            # On essaie d'obtenir le JSON de la réponse
            try:
                resp_json = resp.json()
            except json.JSONDecodeError:
                logger.error(f"Réponse non-JSON:\n{resp.text[:200]}")
                continue
            
            # Affichage d’un extrait du JSON (en DEBUG, car potentiellement verbeux)
            logger.debug(f"Réponse (début): {json.dumps(resp_json, indent=2)[:1000]}...")

            # Nombre de tokens si success
            tokens_used = None
            if resp.status_code == 200:
                if "usage" in resp_json:
                    tokens_used = resp_json["usage"].get("total_tokens")
                logger.info(
                    f"[OK] Réponse HTTP 200 en {elapsed_time:.2f}s, "
                    f"Tokens utilisés={tokens_used if tokens_used else 'N/A'}"
                )
            else:
                # On log au niveau ERROR pour signifier un souci
                logger.error(
                    f"[ERREUR] HTTP {resp.status_code}, texte={resp_json.get('message', resp.text)}"
                )
        except requests.exceptions.Timeout:
            logger.error("Timeout après 30s.")
        except Exception as e:
            logger.exception(f"Exception lors de la requête: {str(e)}")

# On exécute le test brut
test_brut_endpoints()


## 5) Test avec la librairie `openai`

On reproduit un appel classique OpenAI, 
mais en changeant `openai.api_base` et `openai.api_key` 
pour chaque endpoint.


In [ ]:
from openai import OpenAI

def test_openai_chat(api_base, api_key, prompt, model):
    """
    Appel classique OpenAI, en utilisant la classe `OpenAI`
    et en gérant la journalisation via logger.
    """
    # Création du client OpenAI-compatible
    client = OpenAI(
        api_key=api_key,
        base_url=api_base
    )

    if not model:
        logger.error("[!] Modèle non défini.")
        raise ValueError("Modèle non défini")

    try:
        # Appel chat.completions
        logger.debug("Appel de client.chat.completions.create()...")
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_completion_tokens=500
        )
        content = response.choices[0].message.content
        tokens_used = response.usage.total_tokens if response.usage else None
        return content, tokens_used
    except Exception as e:
        logger.exception(f"Exception lors de l'appel OpenAI: {str(e)}")
        return None, None

def test_openai_endpoints():
    """Itère sur tous les endpoints et lance un prompt 'philosophie stoïcienne'."""
    for ep in endpoints:
        label_model = ep.get("model", "<aucun>")
        logger.info(f"\n=== Test openai pour {label_model} ({ep['name']}) ===")
        
        start_time = time.time()
        prompt = "Peux-tu résumer la philosophie stoïcienne en quelques lignes ?"
        content, tks = test_openai_chat(
            ep["api_base"],
            ep["api_key"],
            prompt,
            ep["model"]
        )
        elapsed_time = time.time() - start_time

        if content:
            logger.info(f" -> Réponse (début): {content[:200]}...")
            logger.info(f" -> Nb tokens: {tks}")
            logger.info(f" -> Temps écoulé: {elapsed_time:.2f} sec")
        else:
            logger.warning(" -> Pas de contenu (erreur ou exception).")

# On exécute le test
test_openai_endpoints()


## 6) Test avec Semantic Kernel (optionnel)

Exemple d'intégration avec [Semantic Kernel](https://github.com/microsoft/semantic-kernel).
On crée un Kernel, on y ajoute un service chat OpenAI-like avec l'endpoint souhaité, 
et on exécute un prompt simple.


In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    OpenAIChatPromptExecutionSettings
)
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.functions import KernelArguments
from openai import AsyncOpenAI
import asyncio

async def test_semantic_kernel():
    """
    Exécute un prompt via Semantic Kernel pour chaque endpoint,
    et journalise les résultats en couleur via `logger`.
    """
    for ep in endpoints:
        model_id = ep.get("model")
        api_key = ep["api_key"]

        logger.info(f"=== Test Semantic Kernel pour endpoint='{ep['name']}', model='{model_id}' ===")

        kernel = sk.Kernel()
        async_client = AsyncOpenAI(api_key=api_key, base_url=ep["api_base"])

        kernel.add_service(
            OpenAIChatCompletion(
                service_id="default",
                ai_model_id=model_id,
                async_client=async_client
            )
        )
        logger.debug("Service OpenAI ajouté au Kernel.")

        prompt_template = "Explique ce qu'est l'apprentissage profond (deep learning) en 500 mots."

        exec_settings = OpenAIChatPromptExecutionSettings(
            service_id="default",
            ai_model_id=model_id,
            max_completion_tokens=500,
        )

        pt_config = PromptTemplateConfig(
            template=prompt_template,
            name="deepLearningFunction",
            template_format="semantic-kernel",
            input_variables=[],
            execution_settings=exec_settings,
        )

        func = kernel.add_function(
            function_name="deepLearningFunction",
            plugin_name="defaultPlugin",
            prompt_template_config=pt_config
        )

        try:
            logger.info("  -> Exécution en cours (Semantic Kernel)...")
            start_time = time.time()
            result = await kernel.invoke(func, KernelArguments())
            elapsed = time.time() - start_time

            # Comptage approximatif de tokens
            tokens_count = len(str(result).split())
            speed = tokens_count / elapsed if elapsed > 0 else 0

            logger.info(f"  -> Résultat (début): {str(result)[:400]}...")
            logger.info(f"  -> Durée: {elapsed:.2f}s, Tokens={tokens_count}, speed={speed:.2f} tok/s")
        except Exception as e:
            logger.exception(f"  [!] Erreur SK sur endpoint='{ep['name']}': {str(e)}")


import nest_asyncio
nest_asyncio.apply()

await test_semantic_kernel()


## 9.1) Test du Function/Tool Calling sur chaque endpoint

Avec vLLM, lorsqu’on démarre les containers avec `--enable-auto-tool-choice` et un `--tool-call-parser` adéquat,
le modèle peut déclencher automatiquement un « tool call » s’il juge qu’un outil est pertinent.  
On doit alors inclure un paramètre `tools` dans la requête, et indiquer `tool_choice="auto"` (ou un nom de fonction précis).

**Note** : Pour exécuter concrètement la fonction côté client Python, on doit définir une fonction Python qui correspond,
et réinjecter manuellement le résultat dans la conversation.  
Voici un exemple simplifié : on va appeler un `get_weather(location, unit)` sur *tous* les endpoints.


In [ ]:
import json
import openai
from openai import OpenAI

def get_weather(location: str, unit: str):
    """Exemple de fonction locale pour la météo."""
    return f"Simulation: Météo à {location}, unité={unit}, ciel dégagé."

def test_tool_calling():
    """
    Test du Function/Tool Calling pour chaque endpoint, en mode auto (tool_choice='auto').
    On journalise les étapes et le résultat.
    """
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Obtenir la météo pour un lieu donné",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "Ex: 'Paris, France'"},
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
                    },
                    "required": ["location", "unit"]
                }
            }
        }
    ]

    user_message = "Bonjour, est-ce que tu peux me donner la météo pour Marseille en celsius ?"

    for ep in endpoints:
        logger.info(f"=== Test Tool Calling sur endpoint '{ep['name']}' ===")

        openai.api_base = ep["api_base"]
        openai.api_key  = ep["api_key"]

        client = OpenAI(
            api_key=ep["api_key"],
            base_url=ep["api_base"]
        )

        try:
            logger.debug("Appel chat.completions avec tool_choice='auto'")
            response = client.chat.completions.create(
                model=ep.get("model"),
                messages=[{"role": "user", "content": user_message}],
                tools=tools,
                tool_choice="auto",
            )
        except Exception as ex:
            logger.error(f"[!] Erreur lors de l'appel sur endpoint='{ep['name']}': {ex}")
            continue

        choice = response.choices[0]
        msg = choice.message

        text_content = msg.content if msg.content else ""
        tool_calls = msg.tool_calls

        logger.debug(f"Réponse textuelle (début): {text_content[:400]!r}")
        logger.debug(f"Contenu complet de la réponse: {json.dumps(response.to_dict(), indent=2)}")

        if tool_calls:
            logger.info(f"  -> {len(tool_calls)} tool_call(s) détecté(s) sur endpoint='{ep['name']}'.")
            for call in tool_calls:
                func_name = call.function.name
                args_str  = call.function.arguments or "{}"
                args_dict = json.loads(args_str)

                logger.info(f"     Fonction appelée: {func_name} | arguments={args_dict}")

                # Exécution locale simulée
                if func_name == "get_weather":
                    result = get_weather(**args_dict)
                    logger.info(f"     => Résultat simulé: {result}")
                else:
                    logger.warning(f"     => Fonction inconnue: {func_name}")
        else:
            logger.warning("  -> Aucun tool_call détecté dans la réponse.")

# On l’exécute
test_tool_calling()


## 9.2) Test du mode « Reasoning Outputs »

Certains modèles (p. ex. DeepSeek R1) sont lancés avec `--enable-reasoning --reasoning-parser deepseek_r1`.
Cela permet de renvoyer, en plus du `content` final, un champ `reasoning_content` qui détaille la chaîne de raisonnement.

Voici un exemple d’appel sur *tous* les endpoints (certains n’auront pas de champ `reasoning_content` si le modèle ne supporte pas le raisonnement).


In [ ]:
def test_reasoning():
    """
    Test du mode reasoning_output (champ 'reasoning_content') 
    s'il est activé sur certains modèles. 
    """
    question = "Combien font 253 * 73 - 287 ?"

    for ep in endpoints:
        logger.info(f"=== Test Reasoning Output sur endpoint='{ep['name']}' ===")

        openai.api_base = ep["api_base"]
        openai.api_key  = ep["api_key"]

        client = OpenAI(
            api_key=ep["api_key"],
            base_url=ep["api_base"]
        )

        try:
            response = client.chat.completions.create(
                model=ep.get("model"),
                messages=[{"role": "user", "content": question}]
            )
        except Exception as ex:
            logger.error(f"  [!] Erreur lors de l'appel endpoint='{ep['name']}': {ex}")
            continue

        choice = response.choices[0]
        msg = choice.message

        reasoning_part = getattr(msg, "reasoning_content", None)
        final_content = msg.content or ""

        logger.debug(f"  -> Réponse (finale) : {final_content[:250]}")
        if reasoning_part:
            logger.info(f"  -> Raisonnement   : {reasoning_part[:250]}")
        else:
            logger.warning("  -> Pas de 'reasoning_content' pour ce modèle/parsing.")

# Lancement
test_reasoning()


## 7) Benchmark final (avec journaux réguliers)

Cette étape exécute un warm-up + N itérations par endpoint.
On calcule ensuite la vitesse tokens/s. 

**Important** : Le prompt est un peu plus long, et la génération peut 
prendre du temps selon la taille du modèle ou la quantization.

Pour ne pas paraître figé, on ajoute des `print` avant et après l'appel, 
pour indiquer la progression.


In [ ]:
import sys
import openai
from openai import OpenAI

# Paramètre: combien de fois on répète la requête pour la mesure.
N_REPEATS = 1
all_results = []

def query_once(api_base, api_key, prompt, model=None):
    """
    Appel unique via la nouvelle API (client = OpenAI(...)) + log.
    Retourne (elapsed_seconds, total_tokens).
    """
    client = OpenAI(api_key=api_key, base_url=api_base)

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    start_time = time.time()
    logger.debug("      ... appel client.chat.completions.create() en cours ...")
    try:
        resp = client.chat.completions.create(
            messages=messages,
            model=model,
            max_completion_tokens=1000,
            stream=False,
        )
        elapsed = time.time() - start_time

        tokens = resp.usage.total_tokens if resp.usage else None
        return elapsed, tokens
    except Exception as e:
        logger.error(f"      [!] Exception: {e}")
        return None, None

def run_benchmark(label, prompt, repeats=N_REPEATS):
    """
    1) Warm-up (unique appel) + N appels chronométrés par endpoint.
    2) On log avant/après chaque appel via logger.
    3) On stocke les métriques dans all_results.
    """
    global all_results

    for ep in endpoints:
        logger.info(f"=== Benchmark: endpoint={ep['name']}, label={label} ===")

        # WARM-UP
        logger.info("   (warm-up) Lancement d'un appel simple.")
        w_elapsed, w_tokens = query_once(
            ep["api_base"], ep["api_key"],
            "Warm up. Ignorez ce message.",
            model=ep.get("model", None)
        )
        if w_elapsed is None:
            logger.warning(f"   [!] Warm-up échoué => skip {ep['name']}.")
            continue

        total_time = 0.0
        total_tokens = 0
        success_count = 0

        logger.info(f"   (benchmark) On va faire {repeats} itérations.")
        for i in range(repeats):
            logger.info(f"   -> Iteration {i+1}/{repeats}")
            e, tks = query_once(
                ep["api_base"], ep["api_key"],
                prompt,
                model=ep.get('model', None)
            )
            if e is None:
                logger.error("      Echec de l'appel, on continue avec le suivant.")
                continue
            logger.info(f"      => Durée: {e:.2f}s, tokens={tks}")
            total_time += e
            if tks is not None:
                total_tokens += tks
            success_count += 1

        if success_count == 0:
            logger.warning("   [!] Aucune itération réussie pour ce endpoint.")
            continue

        avg_time = total_time / success_count
        tok_per_sec = 0.0
        if total_time > 0:
            tok_per_sec = total_tokens / total_time

        res = {
            "label": label,
            "endpoint": ep["name"],
            "repeats": repeats,
            "success_count": success_count,
            "total_time_s": total_time,
            "avg_time_s": avg_time,
            "total_tokens": total_tokens,
            "tokens_per_sec": tok_per_sec
        }
        all_results.append(res)
        logger.info(
            f"   => {ep['name']} OK: {success_count}/{repeats} calls, "
            f"avg_time={avg_time:.2f}s, speed={tok_per_sec:.2f} tok/s"
        )

    logger.info(f"[INFO] Fin du benchmark pour label='{label}'.")

# Exemple d’appel:
label_run = "auto_benchmark1"
USER_PROMPT = (
    "Rédige un texte d'environ 1000 mots sur l'IA, "
    "en évoquant l'apprentissage machine, les grands modèles de langage, "
    "et quelques perspectives d'évolution."
)

run_benchmark(label_run, USER_PROMPT, repeats=1)

logger.info(f"\n=== Résultats de ce premier benchmark (label='{label_run}') ===")
for r in all_results:
    if r["label"] == label_run:
        logger.info(
            f"{r['endpoint']} -> {r['success_count']}/{r['repeats']} ok, "
            f"avg time={r['avg_time_s']:.2f}s, speed={r['tokens_per_sec']:.2f} tok/s"
        )



## 8) Relancer le benchmark après mise à jour des containers

Pour passer de Oobabooga à vLLM (ou inversement), 
il faut arrêter le premier groupe de containers 
et démarrer le second (avec `docker-compose.*.yml`).

Ensuite, on ré-exécute ce notebook (ou au moins les cellules de configuration + benchmark).

Ici, on propose un prompt qui demande si tu veux relancer un benchmark 
avec un nouveau label.


In [ ]:
do_rerun = input("Voulez-vous exécuter un nouveau benchmark ? (y/n) ").strip().lower()

if do_rerun == "y":
    rename_label = input("Entrez un label pour le 1er benchmark (actuellement 'auto_benchmark1'): ").strip()
    for r in all_results:
        if r["label"] == "auto_benchmark1":
            r["label"] = rename_label
    second_label = input("Entrez un label pour le 2e benchmark : ").strip()
    run_benchmark(second_label, USER_PROMPT, repeats=N_REPEATS)

logger.info("\n=== Récapitulatif complet de tous les benchmarks ===")
for r in all_results:
    logger.info(
        f"{r['label']} | {r['endpoint']} -> {r['success_count']}/{r['repeats']} ok, "
        f"time={r['avg_time_s']:.2f}s, speed={r['tokens_per_sec']:.2f} tok/s"
    )


## 9) Test de traitement parallèle (batching)

Dans cette cellule, nous allons :
- Définir un nombre de requêtes à envoyer en parallèle (`N_PARALLEL`).
- Pour chaque endpoint, lancer ces requêtes en **concurrence**.
- Mesurer le temps total écoulé et le nombre total de tokens.
- Calculer la **vitesse globale** de traitement (tokens / seconde) lorsque plusieurs requêtes arrivent simultanément.

**vLLM** est réputé supporter le batching token-level et donc bénéficier d'une meilleure latence moyenne et d'un meilleur débit lorsqu'il y a plusieurs requêtes en parallèle.


In [ ]:
import aiohttp
import asyncio

N_PARALLEL = 25
PARALLEL_PROMPT = (
    "Bonjour, ceci est un test de requêtes parallèles. "
    "Peux-tu me donner quelques idées créatives pour un week-end ?"
)

async def async_chat_completion(api_base: str, api_key: str, model: str, prompt: str):
    url = f"{api_base}/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 200
    }

    async with aiohttp.ClientSession() as session:
        start_t = time.time()
        try:
            async with session.post(url, headers=headers, json=payload, timeout=60) as resp:
                elapsed = time.time() - start_t
                if resp.status == 200:
                    data = await resp.json()
                    tokens = None
                    if "usage" in data and data["usage"].get("total_tokens"):
                        tokens = data["usage"]["total_tokens"]
                    return (elapsed, tokens)
                else:
                    return (None, None)
        except Exception as e:
            logger.error(f"[!] Exception asynchrone: {e}")
            return (None, None)

async def run_parallel_test(endpoint, n_parallel, prompt):
    api_base = endpoint["api_base"]
    api_key  = endpoint["api_key"]
    model    = endpoint.get("model", None)

    tasks = []
    for _ in range(n_parallel):
        tasks.append(asyncio.create_task(async_chat_completion(api_base, api_key, model, prompt)))

    start_time = time.time()
    results = await asyncio.gather(*tasks)
    total_time = time.time() - start_time

    nb_ok = 0
    sum_tokens = 0
    for (elapsed, tokens) in results:
        if elapsed is not None and tokens is not None:
            nb_ok += 1
            sum_tokens += tokens

    return {
        "endpoint": endpoint["name"],
        "n_req": n_parallel,
        "n_ok": nb_ok,
        "total_time_s": total_time,
        "sum_tokens": sum_tokens
    }

async def parallel_benchmark(endpoints_list, n_parallel, prompt):
    summary = []
    logger.info(f"==== Lancement du test parallèle: {n_parallel} requêtes simultanées par endpoint ====")
    
    for ep in endpoints_list:
        logger.info(f"=== Parallel test sur '{ep['name']}' ===")
        res = await run_parallel_test(ep, n_parallel, prompt)

        nb_ok = res["n_ok"]
        total_time = res["total_time_s"]
        sum_tokens = res["sum_tokens"]
        speed = sum_tokens / total_time if total_time > 0 else 0

        logger.info(f"  -> {nb_ok}/{n_parallel} appels OK")
        logger.info(f"  -> Durée totale (concurrente): {total_time:.2f} s")
        logger.info(f"  -> Tokens cumulés: {sum_tokens}")
        logger.info(f"  -> Vitesse globale: {speed:.2f} tok/s")

        summary.append(res)

    logger.info("\n=== Récapitulatif du test parallèle ===")
    for s in summary:
        speed = 0.0
        if s["total_time_s"] > 0:
            speed = s["sum_tokens"] / s["total_time_s"]
        logger.info(
            f"{s['endpoint']}: {s['n_ok']}/{s['n_req']} ok, "
            f"total_time={s['total_time_s']:.2f}s, speed={speed:.2f} tok/s"
        )

import nest_asyncio
nest_asyncio.apply()

await parallel_benchmark(endpoints, N_PARALLEL, PARALLEL_PROMPT)
